# First time need to Install the below libraries

## uncomment and install

In [ ]:
#pip install OSMPythonTools overpy ee geemap geopandas

# Also sign-up here https://code.earthengine.google.com/register for an account

# Mounting Google Drive is important

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Importing Libraries

In [ ]:
import ee
import geemap
import os
from OSMPythonTools.overpass import Overpass, overpassQueryBuilder
import geopandas as gpd
import overpy

# Generate a map

### When you run the below code You'll be asked to verify that is also a one time process. Refer the word doc for reference.


In [144]:
m = geemap.Map()
m.setCenter(4.012275294602337, 50.97159227395128,9.7) #setting center to Flanders with 9.7 zoom level
m.add_basemap("HYBRID") #Hybrid here represents google satellite basemap
m

Map(center=[50.97159227395128, 4.012275294602337], controls=(WidgetControl(options=['position', 'transparent_b…

# In-order to capture the longitude and latitude information

In [146]:
#before running this draw a geometry on the map above using drawing tools for the Area of interest
m.draw_features
#m.draw_last_feature
fc = ee.FeatureCollection(m.draw_features) #lat&long values input into a feature

lat_lng_list = []
features = fc.getInfo()['features']
for feature in features:
    geometry = feature['geometry']['coordinates']
    # Extracting coordinates
    if feature['geometry']['type'] == 'Point':
        lat_lng_list.append(geometry[::-1])
    else:
        for coords in geometry:
            if isinstance(coords[0], list):
                for sub_coords in coords:
                    lat_lng_list.append(sub_coords[::-1])
            else:
                lat_lng_list.append(coords[::-1])

# Print or use the lat_lng_list containing latitude and longitude coordinates
print(lat_lng_list)

[[51.025161, 3.720546], [51.026578, 3.720546], [51.026578, 3.72458], [51.025161, 3.72458], [51.025161, 3.720546]]


In [156]:
#This is for a single feature, therefore I've commented it out (For testing purposes only)

"""

# # Define your bounding box coordinates
# bbox = (lat_lng_list[0][0], lat_lng_list[0][1], lat_lng_list[1][0], lat_lng_list[2][1])

# # Create an Overpass object
# overpass = Overpass()

# # Build your query to search for specific features within the bounding box
# query = overpassQueryBuilder(
#     bbox=bbox,
#     elementType=['way'],
#     selector=['"leisure"="park"'],
#     includeGeometry=True,
#     out='body'
# )

# # Fetch the results
# response = overpass.query(query)
# print(response.elements())

"""

[<OSMPythonTools.element.Element object at 0x7d3085d8d630>, <OSMPythonTools.element.Element object at 0x7d3085d8f790>]


# Capturing the Map items

In [157]:
# Define your own bounding box coordinates or let python capture it from above process
#Uncomment and give your own coordinates and comment other one (if you want your own lat lng bounding box)
# bbox = (lat_min, lng_min, lat_max, lat_min)
bbox = (lat_lng_list[0][0], lat_lng_list[0][1], lat_lng_list[1][0], lat_lng_list[2][1])

# Create an Overpass object
overpass = Overpass()

#Define the queries in a list (You can define your own list as well; Refer this https://wiki.openstreetmap.org/wiki/Map_features)

queries = [
    ('"leisure"="park"', 'Parks'),
    ('"landuse"="meadow"', 'Meadows'),
    ('"natural"="tree_row"', 'Tree Rows'),
    ('"landuse"="grass"', 'Grass Areas'),
    ('"landuse"="forest"', 'Forests'),
    ('"landuse"="flowerbed"', 'Flowerbeds'),
    ('"landuse"="orchard"', 'Orchards')
]

# Initialize a dictionary to store response elements for each query
query_responses = {}

# Loop through queries
for query_selector, query_name in queries:
    query = overpassQueryBuilder(
        bbox=bbox,
        elementType=['way'],
        selector=[query_selector],
        includeGeometry=True,
        out='body'
    )
    response = overpass.query(query)
    # Only add to query_responses if response is not empty
    if response.elements():
        query_responses[query_name] = response

## testing

In [161]:
print(query_responses.keys())

dict_keys(['Parks', 'Grass Areas', 'Forests'])


## testing

In [163]:
print(query_responses['Forests'].elements())

# Total count of captured feature

In [159]:
# Initialize total count
total_count = 0

# Loop through each key and sum up the lengths of the lists
for key, value in query_responses.items():
    total_count += len(value.elements())

# Print the total count
print("Total count of all values:", total_count)

Total count of all values: 4


# Visulize one of the features on map to check if the tool is running

In [165]:
roi = ee.Feature(query_responses["Parks"].elements()[0].geometry())
m.addLayer(roi,{'color': 'db0f0fff', 'width': 2, 'lineType': 'solid', 'fillColor': 'e60000a8'},'Grasslands')
m.centerObject(roi,16)
m

Map(bottom=5615038.0, center=[51.02598858853073, 3.7243083781807935], controls=(WidgetControl(options=['positi…

# Downloads all the features as a Shapefile using a for loop

In [183]:
for key, query_response in query_responses.items():
    for index, value in enumerate(query_response.elements()):
        roi = ee.Feature(value.geometry())
        task_name = f'{key}_{index}'  # Generate unique task name using key and index
        task = ee.batch.Export.table.toDrive(
            collection=ee.FeatureCollection([roi]),
            description=task_name,
            folder='earth_engine_demos',
            fileFormat='SHP',
        )
        task.start()